In [ ]:
%config IPCompleter.greedy=True

# SQLAlchemy

In [ ]:
pip install sqlalchemy

In [ ]:
# - SQLAlchemy is a Python toolkit & ORM 
# - SQLAlchemy considers the database to be a relational algebra engine
# - Rows can be selected from tables, joins & other select statements
# - ORM: data mapper pattern where classes can be mapped to the database. This allows the object 
# model & database schema to develop in a decoupled way
# - Companies that use SQLAlchemy: Yelp!, reddit, DropBox, Survey Monkey

# Resource - https://www.sqlalchemy.org/

from sqlalchemy import create_engine, Column, ForeignKey, Integer, String, Table
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker

# The new base class will be given a metaclass that produces appropriate
# objects and makes the appropriate mapper() calls based on the information
# provided declaratively in the class and any subclasses of the class
Base = declarative_base()

class Person(Base):
    __tablename__ = 'person'
    id = Column(Integer, primary_key=True)
    first_name = Column(String(255), nullable=False)
    last_name = Column(String(255), nullable=False)

    def __init__(self, first_name, last_name):
        self.first_name = first_name
        self.last_name = last_name


class Student(Person):
    __tablename__ = 'student'
    id = Column(None, ForeignKey('person.id'), primary_key=True)
    student_id = Column(Integer, nullable=False, unique=True)
    courses = relationship('Course', secondary='student_course')

    def __init__(self, first_name, last_name, student_id):
        super().__init__(first_name, last_name)
        self.student_id = student_id


class Course(Base):
    __tablename__ = 'course'
    id = Column(String, primary_key=True)
    name = Column(String(255), nullable=False)
    semester = Column(Integer, nullable=False)
    students = relationship('Student', secondary='student_course')

    def __init__(self, id, name, semester):
        self.id = id
        self.name = name
        self.semester = semester


class StudentCourse(Base):
    __tablename__ = 'student_course'
    student_id = Column(Integer, ForeignKey(
        'student.student_id'), primary_key=True)
    course_id = Column(String, ForeignKey('course.id'), primary_key=True)


engine = create_engine('sqlite:///students.db', echo=False)
Session = sessionmaker()
Session.configure(bind=engine)
# Issue a CREATE statement for all tables, using the given Connectable for connectivity
Base.metadata.create_all(engine)

session = Session()

records = [
    Student('Salvador', 'Allen', 1000045392),
    Student('James', 'Gillespie', 1000054576),
    Student('Jack', 'Carney', 1000053613),
    Course('IN512', 'Fundamentals of Web Development', 1),
    Course('IN515', 'Introduction to Networks', 2),
    Course('IN615', 'Routing & Switching', 1),
    Course('IN628', 'Programming 4', 2),
    StudentCourse(student_id=1000045392, course_id='IN512'),
    StudentCourse(student_id=1000045392, course_id='IN515'),
    StudentCourse(student_id=1000045392, course_id='IN628'),
    StudentCourse(student_id=1000054576, course_id='IN512'),
    StudentCourse(student_id=1000053613, course_id='IN512'),
]

session.add_all(records)
session.commit()

In [ ]:
def main():
    query = session.query(Course).filter(Course.name == 'Programming 4').one()
    print('==================================================')  
    for student in query.students:
        print(f'{student.first_name} {student.last_name} {student.student_id}')
    print('==================================================')    
    
    query = session.query(Course).join(StudentCourse).join(
        Student).filter(Student.student_id == 1000045392).filter(Course.semester == 2)
    print('==================================================')  
    for course in query:
        print(f'{course.name}')
    print('==================================================')  

    session.close()


if __name__ == '__main__':
    main()